In [1]:
# import dependencies

from collections import Counter
import numpy as np
import nltk
from nltk.corpus import stopwords 
nltk.download("popular")
import re
import sklearn.manifold
import multiprocessing
import pandas as pd
import gensim.models.word2vec as w2v

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

# ***Getting the data and loading it***

In [2]:
# import the data and view it
wine_df = pd.read_csv("final_wine_data_172k_test.csv")
wine_df.head()

,Unnamed: 0,country,description,price,points,variety,winery
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",15.0,87,Portuguese Red,Quinta dos Avidagos
1,2,US,"Tart and snappy, the flavors of lime flesh and...",14.0,87,Pinot Gris,Rainstorm
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",13.0,87,Riesling,St. Julian
3,4,US,"Much like the regular bottling from 2012, this...",65.0,87,Pinot Noir,Sweet Cheeks
4,5,Spain,Blackberry and raspberry aromas show a typical...,15.0,87,Tempranillo-Merlot,Tandem


In [0]:
# split out the variety and description
variety = wine_df['variety']
description = wine_df['description']

# ***Getting the Word2Vec Model ready***

In [0]:
# In order to train a word2vec model, all of the description data will need to be concatenated into one giant string.
corpus_raw = ""
for d in description:
    corpus_raw += d

In [0]:
# create a toeknizer to break the words up
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [0]:
# run the tokenizer with our raw data
raw_sentences = tokenizer.tokenize(corpus_raw)

In [0]:
# Create a function that will grab all of the sentences and make them into words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [0]:
# run the above function to a variable sentences
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [11]:
# view the differences between the raw stentences and the words
print(raw_sentences[234])
print(sentence_to_wordlist(raw_sentences[234]))

The tactile acidity once sipped creates a fascinating soft buzz on the tongue, with flavors of dried mint, brisk raspberry and cherry tomato.
['The', 'tactile', 'acidity', 'once', 'sipped', 'creates', 'a', 'fascinating', 'soft', 'buzz', 'on', 'the', 'tongue', 'with', 'flavors', 'of', 'dried', 'mint', 'brisk', 'raspberry', 'and', 'cherry', 'tomato']


In [12]:
# see how many tokens are in this whole dataset
token_count = sum([len(sentence) for sentence in sentences])
print('The wine corpus contains {0:,} tokens'.format(token_count))

The wine corpus contains 7,203,114 tokens


# ***Training the Word2Vec model***

In [0]:
# creating our vairables for the model and applying the values
num_features = 300
min_word_count = 5
num_workers = multiprocessing.cpu_count()
context_size = 10
downsampling = 1e-3
seed=1993

In [0]:
# creating our word2vec model
wine2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [0]:
# building our word2vec model
wine2vec.build_vocab(sentences)

In [16]:
# seeing how many vocabulary words are in the model
print('Word2Vec vocabulary length:', len(wine2vec.wv.vocab))

Word2Vec vocabulary length: 15406


In [17]:
# getting the total corpus count of the model
print(wine2vec.corpus_count)

305018


In [18]:
# training the model
wine2vec.train(sentences, total_examples=wine2vec.corpus_count, epochs=wine2vec.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(25988037, 36015570)

# ***Playing with the model***

### Word2Vec provides a "most similar" word feature we can use in the model

In [19]:
# run against the model to see what words are similar and what we can predict
wine2vec.most_similar('melon')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('papaya', 0.6805564165115356),
 ('cantaloupe', 0.6621880531311035),
 ('banana', 0.640127420425415),
 ('supremes', 0.6319231986999512),
 ('nectarine', 0.6277747750282288),
 ('honeydew', 0.6268405914306641),
 ('mango', 0.6213712692260742),
 ('peach', 0.6154108047485352),
 ('kiwi', 0.6133053302764893),
 ('tinned', 0.6103756427764893)]

In [20]:
wine2vec.most_similar('berry')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Sunbaked', 0.5775878429412842),
 ('blackberry', 0.572376012802124),
 ('cowhide', 0.557167112827301),
 ('berries', 0.5560652017593384),
 ('horsehide', 0.538925290107727),
 ('conflict', 0.5343519449234009),
 ('withered', 0.5291473865509033),
 ('Porcini', 0.5225446224212646),
 ('Berry', 0.5208200216293335),
 ('aftershocks', 0.5187227725982666)]

In [21]:
wine2vec.most_similar('oak')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('charry', 0.5337623357772827),
 ('Used', 0.5206912755966187),
 ('Cedary', 0.5176864862442017),
 ('muffle', 0.515874981880188),
 ('drums', 0.507997989654541),
 ('roasty', 0.5055288076400757),
 ('oaky', 0.5017535090446472),
 ('cradling', 0.49776583909988403),
 ('compostable', 0.49642103910446167),
 ('staves', 0.495244562625885)]

In [22]:
wine2vec.most_similar('full')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Full', 0.6068189144134521),
 ('supersmooth', 0.5505632758140564),
 ('overflows', 0.5294167995452881),
 ('ultrasmooth', 0.5271635055541992),
 ('gushing', 0.5226176977157593),
 ('abundantly', 0.5214405655860901),
 ('feathered', 0.5092262625694275),
 ('Seriously', 0.5019351840019226),
 ('drenching', 0.5000550150871277),
 ('helpings', 0.4948199987411499)]

In [23]:
wine2vec.most_similar('tannins')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Tannins', 0.6057425141334534),
 ('flexed', 0.5649431943893433),
 ('firm', 0.5524418354034424),
 ('lithely', 0.552230715751648),
 ('buffed', 0.543292760848999),
 ('loosely', 0.540454089641571),
 ('Mouthfilling', 0.5362820029258728),
 ('tannin', 0.5347068905830383),
 ('Sinewy', 0.5340871810913086),
 ('clenching', 0.5325667858123779)]

In [24]:
wine2vec.most_similar('white')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('rennet', 0.5722517967224121),
 ('Kaiser', 0.5488391518592834),
 ('McIntosh', 0.5327709913253784),
 ('Fetel', 0.5320342183113098),
 ('Chamomile', 0.5281394720077515),
 ('salvia', 0.5256830453872681),
 ('Rennet', 0.5230833888053894),
 ('Plumes', 0.5205769538879395),
 ('crystalized', 0.5150020122528076),
 ('Caprettone', 0.5121810436248779)]

In [25]:
wine2vec.most_similar_cosmul("white")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  """Entry point for launching an IPython kernel.


[('rennet', 0.786125123500824),
 ('Kaiser', 0.774418830871582),
 ('McIntosh', 0.7663847804069519),
 ('Fetel', 0.7660163640975952),
 ('Chamomile', 0.7640690207481384),
 ('salvia', 0.7628408074378967),
 ('Rennet', 0.7615410089492798),
 ('Plumes', 0.7602877616882324),
 ('crystalized', 0.7575002908706665),
 ('Caprettone', 0.7560898065567017)]